<a href="https://colab.research.google.com/github/SanjibSarkarU/colab-NN/blob/main/multi_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow-gpu

In [2]:
!pip install pymap3d

In [3]:
import numpy as np
import tensorflow as tf
# from tensorflow.keras.models import Sequential
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.colors as pltc
from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import pymap3d as pm
plt.style.use('dark_background')
colors = [k for k,v in pltc.cnames.items()]

In [28]:
df = pd.read_csv('20220222_153859_hlc.csv', skiprows=1)
df = df.rename(columns={"Latitude (Deg N)": 'lat', "Longitude (Deg W)": 'lng'}, errors="raise")
df = df.loc[(df['lat'] != 0)]
df['h'] = 0
# df.set_index([i for i in range(0, len(df.lat)-1)], inplace=True)
df.reset_index(drop=True, inplace=True)

In [29]:
index = int(len(df.lat)/2)

In [30]:
# index

In [31]:
# df['lat'] = df['lat']-30
# df['lng'] = df['lng'] + 89

In [32]:
' Convert to local coordinate system'
index = index
df['e'], df['n'], df['u'] = pm.geodetic2enu(df.lat, df.lng, df.h, df.lat[index],
                                                    df.lng[index], df.h[index], ell=None, deg=True)


df.head(5)

,Date (mm/dd/yyyy),Time,Num Satellites,Position Uncertainty (m),lat,lng,Speed (kts),Heading (Deg N),Course (Deg N),OCB Temp (C),...,Volts stbd,Amps stbd,charge (%) stbd,StatorTemp (C) stbd,PCBTep (C) stbd,Error Code stbd,h,e,n,u
0,02/22/2022,15:40:32.999,8,3,30.366409,-89.622612,29.89,226.0,269.4,43.2,...,0.0,0.0,0.0,0.0,0.0,0x0,0,585.817265,1578.584509,-0.223042
1,02/22/2022,15:40:33.799,8,3,30.366408,-89.622765,28.79,225.9,269.4,43.3,...,0.0,0.0,0.0,0.0,0.0,0x0,0,571.118054,1578.485065,-0.221685
2,02/22/2022,15:40:35.999,9,3,30.366407,-89.623054,24.93,225.9,269.1,43.2,...,0.0,0.0,0.0,0.0,0.0,0x0,0,543.274052,1578.271902,-0.219202
3,02/22/2022,15:40:37.000,9,3,30.366405,-89.623188,22.74,226.2,269.4,43.3,...,0.0,0.0,0.0,0.0,0.0,0x0,0,530.406116,1578.126043,-0.218083
4,02/22/2022,15:40:37.999,9,3,30.366404,-89.623302,21.17,226.5,269.7,43.3,...,0.0,0.0,0.0,0.0,0.0,0x0,0,519.458852,1578.039046,-0.217161


In [33]:
print(f'Latitude min:{df.e.min()}, Latitude max:{df.e.max()},\nLongitude min:{df.n.min()}, Longitude max:{df.n.max()}')

Latitude min:-204.38509766230544, Latitude max:585.8172653577328,
Longitude min:-106.02933113377932, Longitude max:1578.9230964370831


In [34]:
start = 10500
end = 13700
lat = df.lat[start:end]
lng = df.lng[start:end]

series_time = df.Time[start:end].to_list
series_lat = df.e[start:end]
series_lng = df.n[start:end]

# series_time = df.Time[start:end].to_list
# series_lat = df.lat[start:end]
# series_lng = df.lng[start:end]

coordinates = [[i,j] for i,j in zip(series_lat, series_lng)]
coordinates= np.array(coordinates)
# print(coordinates[:10])

In [35]:
# 'Compare before and after local converstion'
# ' Back to lag lng'
# df['back_lat'], df['back_lng'], _ = pm.enu2geodetic(df.e, df.n, df.u, df.lat[84],
#                                                     df.lng[84], df.h[84], ell=None, deg=True)

In [36]:
# df.head(5)

In [37]:
# type(coordinates)

In [38]:
# Test train split
def train_test_split(dataset, percentage):
    train_size = int(len(dataset) * percentage)
    # test_size = len(dataset) - train_size
    train, test = dataset[0:train_size, :], dataset[train_size:len(dataset), :]
    return train, test


In [39]:
def ts_from_array(data, target, window, future, batch, shuffle):
    data = data[:-(window + 1)] if future != 1 else data
    targets = target
    print("length: ", len(data), len(targets))
    tensor = tf.keras.preprocessing.timeseries_dataset_from_array(data, targets, sequence_length=window,
                                                                  sequence_stride=1,
                                                                  sampling_rate=1, batch_size=batch, shuffle=shuffle,
                                                                  seed=None,
                                                                  start_index=None, end_index=None)
    return tensor

In [40]:
# create scaler
scaler = StandardScaler()
# fit scaler on data
scaler.fit(coordinates)
# apply transform
coordinates = scaler.transform(coordinates)
# print(coordinates[:17])

In [41]:
# print(coordinates[:17])

In [42]:
train, test = train_test_split(coordinates, 0.7)

In [43]:
window_size = 15
future = 2
batch_size = 32

In [44]:
target = train[(window_size + future) - 1:]

In [45]:
xy_train = ts_from_array(train, target, window=window_size, future=future, batch=batch_size, shuffle=False)

length:  2224 2224


In [46]:
def model_lstm(w_size):
    units = 15
    tf.keras.backend.clear_session()
    model = tf.keras.models.Sequential([
        tf.keras.layers.LSTM(w_size + 2, activation='relu', return_sequences=True, input_shape=(w_size, 2)),
        tf.keras.layers.LSTM(units),
        tf.keras.layers.Dense(2, activation='linear')])    
    model.summary()
    # print('Train...', model.summary())
    return model

In [47]:
model = model_lstm(window_size)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 15, 17)            1360      
                                                                 
 lstm_1 (LSTM)               (None, 15)                1980      
                                                                 
 dense (Dense)               (None, 2)                 32        
                                                                 
Total params: 3,372
Trainable params: 3,372
Non-trainable params: 0
_________________________________________________________________


In [48]:
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.0001, momentum=0.9), loss='mse')

In [49]:
epochs = 100
history = model.fit(xy_train, epochs=epochs, verbose=1)

Epoch 1/100


UnknownError: ignored

In [ ]:
# def ts_data_preparation(features, window=5, future=1, batch=1, shuffle=False):
#     target = features[(future - 1):]
#     features = features[:-(future - 1)] if future != 1 else features
#     tensor = TimeseriesGenerator(features, target, length=window, batch_size=batch, shuffle=shuffle)
#     return tensor

In [ ]:
# def test_data_preparation(series, window_size, batch_size):
#     dataset = tf.data.Dataset.from_tensor_slices(series)
#     dataset = dataset.window(window_size, shift=1, drop_remainder=True)
#     dataset = dataset.flat_map(lambda window: window.batch(window_size))
#     # dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:-1], window[-1:]))
#     dataset = dataset.batch(batch_size).prefetch(1)
#     return dataset

In [ ]:
# def train_test_split(dataset, percentage):
#     train_size = int(len(dataset) * percentage)
#     # test_size = len(dataset) - train_size
#     train, test = dataset[0:train_size, :], dataset[train_size:len(dataset), :]
#     return train, test

In [ ]:
# # create scaler
# scaler = StandardScaler()
# # fit scaler on data
# scaler.fit(coordinates)
# # apply transform
# coordinates = scaler.transform(coordinates)

In [ ]:
# coordinates[0:10]

In [ ]:
# # inverse transform
# inverse = scaler.inverse_transform(coordinates)
# inverse[:10]

In [ ]:
# train_coor, test_coor = train_test_split(coordinates, 0.7)

In [ ]:
# train_coor[:10]  #.shape, test_coor.shape

In [ ]:
# window_size = 20
# future = 1
# batch_size = 32

In [ ]:
# train_coor = ts_data_preparation(train_coor, window=window_size, future=future, batch=batch_size, shuffle=True)

In [ ]:
len(train_coor)

In [ ]:
# def model_lstm(w_size):
#     units = 64
#     tf.keras.backend.clear_session()
#     model = tf.keras.models.Sequential([
#         tf.keras.layers.LSTM(units, activation='relu', return_sequences=True, input_shape=(w_size, 2)),
#         tf.keras.layers.LSTM(units, activation='relu'),
#         tf.keras.layers.Dense(2, activation='linear')])
#     model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.0002, momentum=0.9),
#                   loss='mse')
#     model.summary()
#     return model

In [ ]:
# model = model_lstm(window_size)

# epochs = 120

# model.fit(train_coor, epochs=epochs, verbose=1)

In [ ]:
# test_coor = test_data_preparation(coordinates, window_size=window_size, batch_size=batch_size)

In [ ]:
# for example in test_coor:
#   print(example[0].numpy())
#   print(example[1].numpy())
#   break

In [ ]:
predicted_coor = model.predict(test_coor)

In [ ]:
type(predicted_coor)

In [ ]:
# # inverse transform
# predicted_coor = scaler.inverse_transform(predicted_coor)

In [ ]:
# df_n = pd.DataFrame(predicted_coor, columns=('p_lat', 'p_lng'))
# df_n['h'] = 0

In [ ]:
# df_n['p_lat'] = df_n['p_lat']+30
# df_n['p_lng'] = df_n['p_lng']-89

In [ ]:
# df_n.head(10)

In [ ]:
# plt.plot(df_n['p_lng'], df_n['p_lat'], color='r')


In [ ]:
# len(df.lat)

In [ ]:
# ' Back to lag lng'
# df_n['Predted_lat'], df_n['Predicted_lng'], _ = pm.enu2geodetic(df_n.p_lat, df_n.p_lng, df_n.h,
#                                                                                   df.lat[index], df.lng[index],
#                                                                                   df.h[index], ell=None, deg=True)

In [ ]:
# df_n.head(5)

In [ ]:
# plt.plot(df_n.Predicted_lng, df_n.Predted_lat, color='r', label=f'predicted, future:{future}')
# plt.plot(lng, lat, label='Original')
# plt.title(f'Window{window_size}')
# plt.legend()
# plt.show()

In [ ]:
# plt.plot(lng, lat, label='y')

In [ ]:
# plt.plot(df_n.Predicted_lng, df_n.Predted_lat, color='r', label=f'predicted, future:{future}')
# plt.scatter(lng.iloc[len(lng)-1], lat.iloc[len(lng)-1], s=100)